In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import pickle


In [3]:
# Imports
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

In [5]:
import os


model_dir = '/content/drive/Shareddrives/1:1_Krishna_Vijay/Results/Models/'

print(os.listdir(model_dir))

['LSTM_NVDA.h5', 'NVDAscalermodel_GRU', 'best_model_NVDA.h5', 'best_model_NVDA_GRU.h5']


In [5]:
GRU_MODEL_PATH = "/content/drive/Shareddrives/1:1_Krishna_Vijay/Results/best_model_NVDA_GRU.h5"
COMPANY = "NVDA"
train_data= pd.read_csv('/content/drive/Shareddrives/1:1_Krishna_Vijay/Dataset/train_NVDA.csv')
test_data=pd.read_csv('/content/drive/Shareddrives/1:1_Krishna_Vijay/Dataset/test_NVDA.csv')
# Load the saved model
model = tf.keras.models.load_model(GRU_MODEL_PATH)
print("Model loaded.")

Model loaded.


In [6]:
import numpy as np

def prepare_seq(data,seq_len):    #eg seq=len = 50
    sequences,labels = [],[]
    #len of data is 1353 then 1353 - 50 is the range
    for i in range(len(data) - seq_len):
        #eg 0 to 50
        x_temp = data.iloc[i:i+seq_len].values
        #stores 50th value
        y_temp = data.iloc[i+seq_len].values

        sequences.append(x_temp)
        labels.append(y_temp)

    return np.array(sequences), np.array(labels)

Training of model

In [8]:
# define the sequence length
SEQ_LENGTH = 50

# process the dataset
train_sequences,train_labels = prepare_seq(train_data, SEQ_LENGTH)
test_sequences,test_labels = prepare_seq(test_data, SEQ_LENGTH)
print(train_sequences.shape, test_sequences.shape)

(957, 50, 4) (202, 50, 4)


In [1]:
# Make predictions
df_path = '/content/drive/Shareddrives/1:1_Krishna_Vijay/Dataset/df_NVDA.csv'
df = pd.read_csv(df_path)

# Update COLUMNS based on the actual column names in the CSV file
COLUMNS = df.columns.tolist()

#need to fix NOT WORKING
predictions = model.predict(test_sequences)

scaler_path = os.path.join(GRU_MODEL_PATH)
with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)

inverse_predictions = scaler.inverse_transform(predictions)

PRED_COLUMNS = ['open_predicted', 'high_predicted', 'low_predicted', 'close_predicted']
pred_df = pd.DataFrame(inverse_predictions, columns=PRED_COLUMNS)


def evaluations(actual_df, pred_df):
    # Align the DataFrames by index or ensure they are of the same length
    common_length = min(len(actual_df), len(pred_df))
    actual_df = actual_df.iloc[:common_length]
    pred_df = pred_df.iloc[:common_length]

    cols = actual_df.columns.tolist()  # Assuming actual values are in a DataFrame `actual_df`
    for i, column in enumerate(cols):
        pred_col = PRED_COLUMNS[i]
        mse = mean_squared_error(actual_df[column].values, pred_df[pred_col].values)
        print(f"MSE for {column}: {mse}")
        print(f"RMSE for {column}: {np.sqrt(mse)}")

        plt.figure(figsize=(10, 6))
        plt.plot(actual_df.index, actual_df[column], label="Actual")
        plt.plot(pred_df.index, pred_df[pred_col], label="Predicted")
        plt.xlabel("Time")
        plt.ylabel(f"Price")
        plt.title(f"Comparison: Actual vs Predicted {column}")
        plt.legend()
        plt.grid(True)
        plt.show()

# Call the evaluation function
evaluations(df, pred_df)

print(COLUMNS)
print(pred_df.head())
print(inverse_predictions)

NameError: name 'pd' is not defined

creating model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU,Dropout
from tensorflow.keras.optimizers import Adam

# define the learning rate and epochs
LR = 0.001
EPOCHS = 50

# build the model
model = Sequential()
model.add(GRU(units=50,return_sequences = True, input_shape=(train_sequences.shape[1],train_sequences.shape[2])))
model.add(Dropout(0.1))
model.add(GRU(units = 50))
model.add(Dense(4))

# compile the model
model.compile(optimizer=Adam(learning_rate = LR), loss='mean_squared_error', metrics=['mean_squared_error'])

training model

In [ ]:
# train the model
model.fit(train_sequences,train_labels, epochs=EPOCHS, validation_data = (test_sequences,test_labels),verbose = 'auto')

Epoch 1/50
30/30 [==============================] - 6s 78ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0055 - val_mean_squared_error: 0.0055
Epoch 2/50
30/30 [==============================] - 1s 44ms/step - loss: 1.4203e-04 - mean_squared_error: 1.4203e-04 - val_loss: 0.0012 - val_mean_squared_error: 0.0012
Epoch 3/50
30/30 [==============================] - 2s 58ms/step - loss: 1.1664e-04 - mean_squared_error: 1.1664e-04 - val_loss: 0.0016 - val_mean_squared_error: 0.0016
Epoch 4/50
30/30 [==============================] - 1s 49ms/step - loss: 9.7287e-05 - mean_squared_error: 9.7287e-05 - val_loss: 0.0019 - val_mean_squared_error: 0.0019
Epoch 5/50
30/30 [==============================] - 2s 75ms/step - loss: 9.0750e-05 - mean_squared_error: 9.0750e-05 - val_loss: 0.0013 - val_mean_squared_error: 0.0013
Epoch 6/50
30/30 [==============================] - 2s 66ms/step - loss: 8.8795e-05 - mean_squared_error: 8.8795e-05 - val_loss: 0.0041 - val_mean_squared_error: 0

Save best model

In [ ]:
import os
COMPANY = 'NVDA'
SAVE = True
LSTM_MODEL_PATH = "/content/drive/Shareddrives/1:1_Krishna_Vijay/Results"

# save the model if needed
if SAVE:
    model.save(os.path.join(LSTM_MODEL_PATH, f"best_model_{COMPANY}_GRU.h5"))
    print("Model is saved")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model is saved


Making predictions

In [9]:
# make predictions
predictions = model.predict(test_sequences)
# inverse tranform the predictions
inverse_predictions = scaler.inverse_transform(predictions)

7/7 [==============================] - 1s 19ms/step


NameError: name 'scaler' is not defined

making eval function

In [ ]:
# create a dataframe frame using model predictions
PRED_COLUMNS = ['open_predicted','high_predicted','low_predicted','close_predicted']
# create the dataframe
pred_df = pd.concat([df.iloc[-predictions.shape[0]:], pd.DataFrame(inverse_predictions, columns = PRED_COLUMNS, index = df.iloc[-predictions.shape[0]:].index)], axis = 1)
pred_df[COLUMNS] = scaler.inverse_transform(pred_df[COLUMNS])
pred_df.head()

In [ ]:
from sklearn.metrics import mean_squared_error

def evaluations(cols, dataframe = pred_df):
    for column in cols:
        print("{}".format(column))
        pred_col = column.lower() + "_predicted"
        mse = mean_squared_error(dataframe[column].values, dataframe[pred_col].values)
        print("Max and Min: {} and {}".format(dataframe[column].max(), dataframe[column].min()))
        print("MSE:", mse)
        print("RMSE:{}\n".format(np.sqrt(mse)))

        plt.figure(figsize = (10,6))
        plt.plot(dataframe[column], label = "actual")
        plt.plot(dataframe[pred_col], label = "predicted")
        plt.xlabel("Time")
        plt.ylabel(f"Actual and Predicted:{column}")
        plt.title(f"Actual and Predictions Comparison: {column}")
        plt.legend(loc="best")
        plt.grid(True)
        plt.show()
        print("\n\n")

evaluating

In [ ]:
# make evaluations
evaluations(COLUMNS)